In [0]:
%run  "./stocks_functions_helper"

In [0]:
dbutils.widgets.text('folderpath', '/mnt/fopart', 'Folder Path')
dbutils.widgets.text('filename', 'some_file', 'FileName')

yep

Out[11]: 9

In [0]:
# tfolderpath='fopart/year=2020/month=04'
# tfilename= 'fo01APR2020bhav.csv.gz'
# yearpos=tfolderpath.index('year=')
# print(tfolderpath[yearpos+5:yearpos+9])
# yr = tfolderpath[yearpos+5:yearpos+9]
# monpos = tfolderpath.index('month=')
# mnth = tfolderpath[monpos+6:monpos+8]
# daypos = tfilename.index('fo')
# dy = tfilename[daypos+2:daypos+4]
# print(yr, mnth, dy)

2020
2020 04 01

In [0]:
v_folderpath = dbutils.widgets.get('folderpath')
v_filename = dbutils.widgets.get('filename')
print(f'folderrpath: {v_folderpath}, filename: {v_filename}')

folderrpath: /mnt/fopart, filename: some_file

In [0]:
yearpos=v_folderpath.index('year=')
yr = v_folderpath[yearpos+5:yearpos+9]
monpos = v_folderpath.index('month=')
mnth = v_folderpath[monpos+6:monpos+8]
daypos = v_filename.index('fo')
dy = v_filename[daypos+2:daypos+4]

In [0]:
fo_data_location = f'/mnt/{v_folderpath}/{v_filename}'
print(fo_data_location)

fo_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv(fo_data_location)\
    .withColumn('year', lit(yr).cast('int'))\
    .withColumn('month', lit(mnth).cast('int'))\
    .withColumn('day', lit(dy).cast('int'))

/mnt/fopart/year=2020/month=1/day=6

In [0]:
fo_df.show(5)

In [0]:
fodf_grouped_instrument = group_df(fo_df)

In [0]:
fodf_pcr = add_pcr_to_df(fodf_grouped_instrument)

In [0]:
fodf_processed = combine_grouped_and_pcr_dfs(
    fodf_grouped_instrument,fodf_pcr)

In [0]:
spark.conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

In [0]:
print('number of rows in fo processed table for the selected dates before executing')
spark.sql(f'''
select count(*) as dayreccount from fopr where year = {yr} 
and month = {mnth} 
and day = {dy}
''').show()

number of rows in fo processed table for the selected dates before executing
+-----------+
dayreccount|
+-----------+
 1363|
+-----------+

In [0]:
fodf_processed.write.mode('overwrite').insertInto('fopr')

In [0]:
print('number of rows in fo processed table after the partition created for the dates in question')
spark.sql(f'''
select count(*) as dayreccount from fopr where year = {yr} 
and month = {mnth} 
and day = {dy}
''').show()

number of rows in fo processed table after the partition created for the dates in question
+-----------+
dayreccount|
+-----------+
 1363|
+-----------+

In [0]:
spark.sql(f'''
select * from fopr where year = {yr} 
and month = {mnth} 
and day = {dy}
and symbol = 'NIFTY'
''').show(5, False)

+-------------------+------+-----------+----------+----------+---------+-------------+---------+------------------+----+-----+---+
rts |SYMBOL|EXPIRY_DT |INSTRUMENT|OPTION_TYP|CONTRACTS|VALUE |OPEN_INT |PCR |YEAR|MONTH|DAY|
+-------------------+------+-----------+----------+----------+---------+-------------+---------+------------------+----+-----+---+
2020-01-06 00:00:00|NIFTY |09-Jan-2020|OPTIDX |CE |2250050.0|2.063739906E7|1.78911E7|1.0018057376502745|2020|1 |6 |
2020-01-06 00:00:00|NIFTY |20-Feb-2020|OPTIDX |CE |8.0 |78.13 |600.0 |0.0 |2020|1 |6 |
2020-01-06 00:00:00|NIFTY |20-Feb-2020|OPTIDX |PE |0.0 |0.0 |0.0 |0.0 |2020|1 |6 |
2020-01-06 00:00:00|NIFTY |05-Mar-2020|OPTIDX |CE |0.0 |0.0 |0.0 |null |2020|1 |6 |
2020-01-06 00:00:00|NIFTY |25-Jun-2020|OPTIDX |CE |47.0 |450.76 |1661100.0|4.425531914893617 |2020|1 |6 |
+-------------------+------+-----------+----------+----------+---------+-------------+---------+------------------+----+-----+---+
only showing top 5 rows

In [0]:
dbutils.notebook.exit('success')

success